### Run for CPU

In [ ]:
from psutil import *
# This code will return the number of CPU
print("Number of CPU: ", cpu_count())
# This code will return the CPU info
!cat /proc/cpuinfo

In [ ]:
import tensorflow as tf
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras import Input


def preprocess_data(dataset_name):
    if dataset_name == 'mnist':
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        x_train = x_train.reshape(-1, 28, 28, 1)
        x_test = x_test.reshape(-1, 28, 28, 1)
    elif dataset_name == 'fashion':
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
        x_train = x_train.reshape(-1, 28, 28, 1)
        x_test = x_test.reshape(-1, 28, 28, 1)
    elif dataset_name == 'cifar10':
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    else:
        raise ValueError("Invalid dataset name")

    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    return x_train, y_train, x_test, y_test

def build_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate(dataset_name, epochs):
    print(f"\n--- Dataset: {dataset_name.upper()} | Epochs: {epochs} ---")
    x_train, y_train, x_test, y_test = preprocess_data(dataset_name)
    model = build_cnn_model(x_train.shape[1:])

    start_time = time.time()
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=64, validation_split=0.1, verbose=0)
    end_time = time.time()

    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    elapsed_time = end_time - start_time

    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Time taken (seconds): {elapsed_time:.2f}")
    return test_accuracy, elapsed_time

# Main run for CPU or GPU (set via Colab Runtime settings)
datasets = ['mnist', 'fashion', 'cifar10']
epoch_list = [5, 10]


results = {}

for dataset in datasets:
    print(f"\n=== Starting experiments for {dataset.upper()} ===")
    results[dataset] = {}
    for ep in epoch_list:
        print(f"--> Training for {ep} epochs...")
        acc, t = train_and_evaluate(dataset, ep)
        results[dataset][f"{ep}_epochs"] = {'accuracy': acc, 'time': t}


print("\n--- Summary of Results ---")
for ds in results:
    print(f"\nDataset: {ds.upper()}")
    for ep_key in results[ds]:
        acc = results[ds][ep_key]['accuracy']
        time_taken = results[ds][ep_key]['time']
        print(f"{ep_key} → Accuracy: {acc:.4f}, Time: {time_taken:.2f} sec")


In [ ]:
# prompt: add graph to visualise

import numpy as np
import matplotlib.pyplot as plt

# Prepare data for plotting
dataset_names = list(results.keys())
epochs_for_plot = epoch_list

accuracy_data = {epoch: [] for epoch in epochs_for_plot}
time_data = {epoch: [] for epoch in epochs_for_plot}

for dataset in dataset_names:
    for epoch in epochs_for_plot:
        epoch_key = f"{epoch}_epochs"
        if epoch_key in results[dataset]:
            accuracy_data[epoch].append(results[dataset][epoch_key]['accuracy'])
            time_data[epoch].append(results[dataset][epoch_key]['time'])
        else:
             # Handle cases where data might be missing (though in this script it won't be)
             accuracy_data[epoch].append(0)
             time_data[epoch].append(0)


# Create plots
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 12))

bar_width = 0.25
x = np.arange(len(dataset_names))

# Accuracy Plot
ax1 = axes[0]
for i, epoch in enumerate(epochs_for_plot):
    ax1.bar(x + i*bar_width, accuracy_data[epoch], bar_width, label=f'{epoch} Epochs')

ax1.set_ylabel('Test Accuracy')
ax1.set_title('Test Accuracy vs. Dataset and Epochs - CPU')
ax1.set_xticks(x + bar_width * (len(epochs_for_plot) - 1) / 2)
ax1.set_xticklabels(dataset_names)
ax1.legend()
ax1.grid(axis='y')

# Time Plot
ax2 = axes[1]
for i, epoch in enumerate(epochs_for_plot):
    ax2.bar(x + i*bar_width, time_data[epoch], bar_width, label=f'{epoch} Epochs', color=f'C{i}') # Use same colors as accuracy plot

ax2.set_ylabel('Time Taken (seconds)')
ax2.set_title('Training and Evaluation Time vs. Dataset and Epochs - CPU')
ax2.set_xticks(x + bar_width * (len(epochs_for_plot) - 1) / 2)
ax2.set_xticklabels(dataset_names)
ax2.legend()
ax2.grid(axis='y')

plt.tight_layout()
plt.savefig('accuracy_time_plot_cpu.png')
plt.show()

### Run for GPU

In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras import Input


def preprocess_data(dataset_name):
    if dataset_name == 'mnist':
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        x_train = x_train.reshape(-1, 28, 28, 1)
        x_test = x_test.reshape(-1, 28, 28, 1)
    elif dataset_name == 'fashion':
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
        x_train = x_train.reshape(-1, 28, 28, 1)
        x_test = x_test.reshape(-1, 28, 28, 1)
    elif dataset_name == 'cifar10':
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    else:
        raise ValueError("Invalid dataset name")

    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    return x_train, y_train, x_test, y_test

def build_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate(dataset_name, epochs):
    print(f"\n--- Dataset: {dataset_name.upper()} | Epochs: {epochs} ---")
    x_train, y_train, x_test, y_test = preprocess_data(dataset_name)
    model = build_cnn_model(x_train.shape[1:])

    start_time = time.time()
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=64, validation_split=0.1, verbose=0)
    end_time = time.time()

    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    elapsed_time = end_time - start_time

    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Time taken (seconds): {elapsed_time:.2f}")
    return test_accuracy, elapsed_time

# Main run for CPU or GPU (set via Colab Runtime settings)
datasets = ['mnist', 'fashion', 'cifar10']
epoch_list = [5, 10]


results = {}

for dataset in datasets:
    print(f"\n=== Starting experiments for {dataset.upper()} ===")
    results[dataset] = {}
    for ep in epoch_list:
        print(f"--> Training for {ep} epochs...")
        acc, t = train_and_evaluate(dataset, ep)
        results[dataset][f"{ep}_epochs"] = {'accuracy': acc, 'time': t}


print("\n--- Summary of Results - GPU ---")
for ds in results:
    print(f"\nDataset: {ds.upper()}")
    for ep_key in results[ds]:
        acc = results[ds][ep_key]['accuracy']
        time_taken = results[ds][ep_key]['time']
        print(f"{ep_key} → Accuracy: {acc:.4f}, Time: {time_taken:.2f} sec")


In [ ]:
# prompt: add graph to visualise

import numpy as np
import matplotlib.pyplot as plt

# Prepare data for plotting
dataset_names = list(results.keys())
epochs_for_plot = epoch_list

accuracy_data = {epoch: [] for epoch in epochs_for_plot}
time_data = {epoch: [] for epoch in epochs_for_plot}

for dataset in dataset_names:
    for epoch in epochs_for_plot:
        epoch_key = f"{epoch}_epochs"
        if epoch_key in results[dataset]:
            accuracy_data[epoch].append(results[dataset][epoch_key]['accuracy'])
            time_data[epoch].append(results[dataset][epoch_key]['time'])
        else:
             # Handle cases where data might be missing (though in this script it won't be)
             accuracy_data[epoch].append(0)
             time_data[epoch].append(0)


# Create plots
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 12))

bar_width = 0.25
x = np.arange(len(dataset_names))

# Accuracy Plot
ax1 = axes[0]
for i, epoch in enumerate(epochs_for_plot):
    ax1.bar(x + i*bar_width, accuracy_data[epoch], bar_width, label=f'{epoch} Epochs')

ax1.set_ylabel('Test Accuracy')
ax1.set_title('Test Accuracy vs. Dataset and Epochs - GPU')
ax1.set_xticks(x + bar_width * (len(epochs_for_plot) - 1) / 2)
ax1.set_xticklabels(dataset_names)
ax1.legend()
ax1.grid(axis='y')

# Time Plot
ax2 = axes[1]
for i, epoch in enumerate(epochs_for_plot):
    ax2.bar(x + i*bar_width, time_data[epoch], bar_width, label=f'{epoch} Epochs', color=f'C{i}') # Use same colors as accuracy plot

ax2.set_ylabel('Time Taken (seconds)')
ax2.set_title('Training and Evaluation Time vs. Dataset and Epochs - GPU')
ax2.set_xticks(x + bar_width * (len(epochs_for_plot) - 1) / 2)
ax2.set_xticklabels(dataset_names)
ax2.legend()
ax2.grid(axis='y')

plt.tight_layout()
plt.savefig('accuracy_time_plot_gpu.png')
plt.show()